# Forensics - Registry Reader

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Required Packages:__<br>
__OS Supported:__<br>
    -  Linux
__Platforms Supported:__<br>
    -  Azure Notebooks DSVM
__Data Source Required:__<br>
    -  no
    
### Description
The notebook provides sample code to parse registry using regipy on https://github.com/chapinb/pythonforensics.

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Prerequisite check

In [ ]:
# Install only once for DSVM
#!pip install python-registry

## Table of Contents

1. Initialize a hive 
2. Walking down the hierarchy
3. Display current key values
4. Compare Current and LastKnownGood for a key

## 1. Initialize a hive

In [ ]:
# user input i.e.: /home/[[user ID]]/notebooks/downloads/[[Hive]]
hive_path = input('Hive Path:')

In [ ]:
from Registry import Registry
from Registry.RegistryParse import ParseException
class RegistryHelper(object):
    def __init__(self, hive_path):
        self.hive = Registry.Registry(hive_path)
        self.root = self.hive.root()
        self.parent_key = None
        self.current_key = self.root
        
    def get_subkey_name(self, key, subkey_name):
        if key is not None and key.subkeys_number() != 0: 
            return [x.name() for x in key.subkeys() if x.name() == subkey_name]
        
    def get_subkey_name_list(self, key):
        if key is None:
            return []
        elif key is not None and key.subkeys_number() != 0: 
            return [k.name() for k in key.subkeys()]
    
    def get_key_by_name(self, parent_key, key_name):
        if parent_key is None or (parent_key == self.root and key_name is None):
            return self.root
        else:
            keys = [k for k in parent_key.subkeys() if k.name() == key_name]
            if keys is not None and len(keys) > 0:
                return next(iter(keys))
            else:
                return None
    
    def set_keys(self, current_key, selected_key_name):
        self.parent_key = current_key
        self.current_key = self.get_key_by_name(self.parent_key, selected_key_name)
    
    def display_key_properties(self, key):
        if key.values_number() != 0:  
            value_names = {x.name(): x.value() for x in key.values()}
            print("{} - key values:".format(key.path()))
            for name, value in value_names.items():
                print("\t{}: {}".format(name, value))
        else:
            print('No values')
    
    def get_current_key_valuetype_and_value(self, value_name):
        val = self.current_key.value(value_name)
        return val.value_type_str(), val.value()
            
    def compare_current_lastknowngood(self, path):
        select = self.hive.open('Select')
        current = self.hive.open(path.format(select.value("Current").value())) 
        lastknown = self.hive.open(path.format(select.value("LastKnownGood").value()))
        
        ccs_dict = dict()
        lkg_dict = dict()
        key_diff = dict()
        val_diff = dict()
        
        # Collect data from current
        for sub in current.subkeys():
            ccs_l = list()
            for val in sub.subkeys():
                ccs_l.append(val.name())
            ccs_dict[sub.name()] = ccs_l

        # Collect data from LKG
        for sub in lastknown.subkeys():
            lkg_l = list()
            for val in sub.subkeys():
                lkg_l.append(val.name())
            lkg_dict[sub.name()] = lkg_l
        
        # Perform comparison 
        for key in ccs_dict.keys():
            if key not in lkg_dict.keys():
                key_diff[key] = ccs_dict[key]
        
        return key_diff

In [ ]:
# Initialize Helper class
reg_helper = RegistryHelper(hive_path)

## 2. Walking down the hierarchy

In [ ]:
# Select a subkey from root
subkey_list = reg_helper.get_subkey_name_list(reg_helper.current_key)
import ipywidgets as widgets
from IPython.display import display
if subkey_list is not None and len(subkey_list) > 0:
    selected_subkey = widgets.Dropdown(options=subkey_list, value=subkey_list[0],description='subkey list:')
    display(selected_subkey)
else:
    display('No Data')

In [ ]:
# reset parent key and current key
reg_helper.set_keys(reg_helper.current_key, selected_subkey.value)
if reg_helper.current_key is not None:
    display('Walking path: {0}'.format(reg_helper.current_key.path()))

## 3. Display current key values

In [ ]:
reg_helper.display_key_properties(reg_helper.current_key)

In [ ]:
val_type, val = reg_helper.get_current_key_valuetype_and_value('AMLIGlobalHeapSize')
print(val_type)
print(val)

## 4. Compare Current and LastKnownGood for a key

In [ ]:
diff = reg_helper.compare_current_lastknowngood(r'ControlSet{:03d}\Control')

In [ ]:
import pprint
print("======== Different ========")
pprint.pprint(diff)